# Scraping Intro Homework: Wikipedia Table

In this assignment, we'll be extracting data from Wikipedia's table of the tallest buildings in Brooklyn: https://en.wikipedia.org/wiki/List_of_tallest_buildings_in_Brooklyn

### 0) Setup

Import `requests`, `BeautifulSoup`, and `pandas`. Although this homework uses `BeautifulSoup`, you can choose to use `lxml` instead, if you prefer.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

### 1) Grab the HTML for the webpage linked above

Use `requests` to get the HTML, assigning it to a variable

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_tallest_buildings_in_Brooklyn"
html = requests.get(url).text

### 2) Convert the HTML into a `BeautifulSoup` object

In [3]:
soup = BeautifulSoup(html)

### 3) Use `.select(...)` (and `[0]`) to select the main table

That's the one directly under the "Tallest buildings" heading.

Print out the first 100 characters of text from the table to make sure you have the right one.

In [4]:
table = soup.select("table.wikitable")[0]
print(table.text[:100])



Rank

Name[a]

Image

Heightft (m)

Floors

Year completed

Notes


1

The Brooklyn Tower



1,073


### 4) Use `.select(...)` (and `[0]`) again to select the table's first row

... which is its header. (Reminder that the `<thead>` tag is optional. Wikipedia tables don't use it.)

In [5]:
header_el = table.select("tr")[0]

### 5) Extract the column names from that header

Use `.strip()` to remove any leading or trailing whitespace from the names.

First, try doing this with a standard `for` loop:

In [6]:
columns = []
for cell in header_el.select("th"):
     columns.append(cell.text.strip())
        
columns

['Rank',
 'Name[a]',
 'Image',
 'Heightft (m)',
 'Floors',
 'Year completed',
 'Notes']

Now try to do it with a list comprehension:

In [7]:
columns = [ cell.text.strip()
    for cell in header_el.select("th") ]

columns

['Rank',
 'Name[a]',
 'Image',
 'Heightft (m)',
 'Floors',
 'Year completed',
 'Notes']

### 6) Select all non-header row *elements* from the table

Since the header was the first row, you'll want to skip that one. How many rows are there? (Check with your eyes that this number matches what you deduce from the rankings column in the browser-rendered table.)

In [8]:
row_els = table.select("tr")[1:]
len(row_els)

73

### 7) For each row, extract the text of each cell into a Python list

First, try this as two nested `for` loops:

In [9]:
rows = []

for row in row_els:
    cells = []
    for cell in row.select("td"):
        cells.append(cell.text.strip())
    rows.append(cells)
    
rows[:2]

[['1',
  'The Brooklyn Tower',
  '',
  '1,073 (327)',
  '93',
  '2022',
  'Topped out in October 2021.[2][22][23][24]'],
 ['2',
  'Brooklyn Point',
  '',
  '720 (219)',
  '68',
  '2019',
  "The final phase of Extell's City Point development; topped out in April 2019, it is now the second tallest building in Brooklyn.[25] Also known as 138 Willoughby Street,[26][27] 1 City Point,[28] and City Point Tower III.[28][29][30]"]]

Now try with two list comprehensions, one nested in the other:

In [10]:
rows = [
    [ cell.text.strip() for cell in row.select("td") ]
    for row in row_els ]

rows[:2]

[['1',
  'The Brooklyn Tower',
  '',
  '1,073 (327)',
  '93',
  '2022',
  'Topped out in October 2021.[2][22][23][24]'],
 ['2',
  'Brooklyn Point',
  '',
  '720 (219)',
  '68',
  '2019',
  "The final phase of Extell's City Point development; topped out in April 2019, it is now the second tallest building in Brooklyn.[25] Also known as 138 Willoughby Street,[26][27] 1 City Point,[28] and City Point Tower III.[28][29][30]"]]

### 8) Turn the data you've extracted into a `pandas` `DataFrame`

In [11]:
buildings = pd.DataFrame(rows, columns=columns)
buildings.head()

,Rank,Name[a],Image,Heightft (m),Floors,Year completed,Notes
0,1,The Brooklyn Tower,,"1,073 (327)",93,2022,Topped out in October 2021.[2][22][23][24]
1,2,Brooklyn Point,,720 (219),68,2019,The final phase of Extell's City Point develop...
2,3,11 Hoyt,,626 (191),51,2020,Topped out in June 2019.[31] A redevelopment o...
3,4,The Hub,,611 (186),52,2017,Also known as 333 Schermerhorn Street. Topped ...
4,5,AVA DoBro,,596 (182),58,2015,"Also known as 100 Willoughby Street, Avalon Wi..."


### 9) Which years are represented by at least 5 buildings?

In [12]:
(
    buildings
    ["Year completed"]
    .value_counts()
    .loc[lambda x: x >= 5]
)

Year completed
2022    11
2020     6
2016     6
Name: count, dtype: int64

### 10) How many total floors do all the buildings have, combined?

In [13]:
(
    buildings
    ["Floors"]
    .astype(int)
    .sum()
)

2603

### 11) How many of the buildings have their own Wikipedia page?

For this, you'll need to query the row elements again; the information won't have been extracted into your `DataFrame`. (Hint: Whether a building has its own Wikipedia page isn't an explicit piece of data, but something you can infer from the presence of a particular sub-element.)

In [14]:
second_cells = [ row.select("td")[1] for row in row_els ]

sum(len(cell.select("a")) > 0 for cell in second_cells)

17

### 12) How many have an image?

You could do this by testing for the presence of another element:

In [15]:
third_cells = [ row.select("td")[2] for row in row_els ]
sum(len(cell.select("img")) > 0 for cell in third_cells)

56

Or through information that's already in your `DataFrame`:

In [16]:
(buildings["Image"] != "—").sum()

56

### Bonus challenge

If we tried to run the same code on https://en.wikipedia.org/wiki/List_of_tallest_buildings_in_New_York_City instead, the results wouldn't be quite right. Try it. Then, examining the HTML of that page, try to figure out why.

If you want an extra-extra challenge, try writing code that would parse that table correctly.

In [17]:
# The answer is that the NYC-wide table uses the "rowspan" attribute,
# for example in the rows for "Deutsche Bank Center North Tower"
# and "Deutsche Bank Center South Tower". This leads the second of those
# pairs of rows to *not* have the full set of cells, causing our code
# to put their cells in the wrong columns.

---

---

---